# Modular RAG  - Raptor RAG (2024, Stanford Univ.)

- Tackling Problem: RAG에 들어가는 문서 수가 많아질수록, 그리고 답안에 필요한 정보량이 long-tail로 방대해야 할때 Retrieval 성능이 너무 좋지 않아진다

- 제안 방법론: 정보들을 단계적 추상화를 통해 enrich시켜서 질문에서 요구하는 context의 뎁스에 알맞는 청크가 뽑히도록 하자.

- (인덱싱 프로세스) Text Chunk를 의미적으로 유사한 다른 Chunk들과 군집화 시키고 -> 군집별 요약문 생성하여 정보 압축 -> 요약문에 대한 임베딩 생성 (전체 프로세스 n번 반복하여 계층적 트리구조 생성)

- (검색 프로세스) Query와 가장 유사한 정보를 Root Node(Layer)에서 찾고, 그 root node가 생성한 요약문이 기반으로 두는 child summary를 찾고, 반복하며 최종적으로 사용자가 지정한 n개의 top-k 유사한 노드가 뽑힐때까지 트리 구조를 traverse함.

# RAPTOR RAG 구조

### Indexing 구조

- 특이점은, GMM기반의 군집화를 하기 때문에 클러스터별 해당하는 노드의 개수가 다르고, 각 차일드노드가 속할 수 있는 패런트노드도 1대다수 관계일 수 있는데, 정보체계라는걸 생각해보면 이 클러스터링 알고리즘이 굉장히 적합한 것으로 사료됨.

# Modular RAG(RAPTOR) 간단구성

- 관련 패키지 임포트
- 활용 LLM api정보 설정
- RAG 파이프라인 글로벌 세팅 설정
- 기준 데이터셋 로드
- 기준 데이터셋 전처리
- (베이스RAG)벡터스토어인덱스 설정
- (베이스RAG)쿼리 엔진 설정
- (RAPTOR RAG) 크로마DB 설정
- (RAPTOR RAG) 쿼리 엔진 설정

In [ ]:
!pip install llama_index OpenAI chromadb datasets llama-index-vector-stores-chroma llama-index-packs-raptor

In [ ]:
# API 설정
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 관련 패키지 임포트
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex
)
import os
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.packs.raptor import RaptorPack
from llama_index.core.query_engine import RetrieverQueryEngine
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# RAG 파이프라인 글로벌 설정
Settings.llm=OpenAI(model='gpt-4o-mini',temperature=0)
Settings.embed_model=OpenAIEmbedding(model='text-embedding-3-small')

In [ ]:
# 데이터 로드
dataset = SimpleDirectoryReader(input_files=[]).load_data()

In [ ]:
# 청크사이즈 전처리 파이프라인 생성
pipeline_200 = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=, chunk_overlap=)
    ]
)

# 설정한 파이프라인 설정으로 기준 다큐먼트 노드 오브젝트로 변경
nodes_200 =

In [ ]:
#기준 데이터 확인
nodes_200

# Naive-RAG 로 베이스 모델 구축

In [ ]:
# 벡터스토어 인덱스 설정
vector_index_200 =

# 쿼리 엔진 설정
query_engine_200 =

In [ ]:
from pprint import pprint

In [ ]:
#질문: "How does Cinderella find a happy ending?"
response =


In [ ]:
pprint(response.response)

In [ ]:
# hallucination이 없을까?
# 참조 컨텍스트 확인
pprint()
pprint()

In [ ]:
# 일단 retrieved context로만 답하게 만들어보자
response = query_engine_200.query(""" You have a long term memory issue. You are not able to answer anything unless the context is provided. When user asks the question, you will be provided with 2 context chunks that are relevant to answer the questions.
                                  answer the question based on the context provided strictly.

                                  Question:
                                  How does Cinderella find a happy ending?
                                  Answer:
                                  """)

In [ ]:
pprint(response.response)

In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

# RAPTOR RAG로 QA 성능 고도화

In [ ]:
# RAPTOR RAG 구현용 크로마DB 활용
client = chromadb.PersistentClient()
collection = client.get_or_create_collection()

vector_store = ChromaVectorStore()

In [ ]:
# 라마인덱스 랩터팩(통합팩) 사용하여 아키 세팅
raptor_pack = RaptorPack(
    , #llamaindex document
    embed_model=, #cluster별 summary text 임베딩 시 활용 모델
    llm=, #Cluster별 Summary Text 생성할 LLM
    vector_store=vector_store, #기반 벡터스토어
    similarity_top_k=2, #추후 Retrieve 시 옵션
    mode="collapsed", #Tree구조, Collapse구조 선택
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ],
)

In [ ]:
# 생성된 인덱스 기반 Raptor Retriever 엔진 정의
raptor_retriever =

raptor_query_engine = RetrieverQueryEngine.from_args(

)

In [ ]:
#질문: "How does Cinderella find a happy ending?"
response =

In [ ]:
pprint(response.response)

In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

# DO IT YOURSELF
- korean_webtext.csv 데이터 사용
- RaptorRAG Pipeline 구성

In [ ]:
import pandas as pd
data = pd.read_csv('<경로>').iloc[:,1:]

# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
documents = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    documents.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
# RAPTOR RAG 구현용 크로마DB
client =
collection =

vector_store =

In [ ]:
# 라마인덱스 랩터팩(통합팩) 사용하여 아키 세팅
raptor_pack = RaptorPack(
    , #llamaindex document
    embed_model=, #cluster별 summary text 임베딩 시 활용 모델
    llm=, #Cluster별 Summary Text 생성할 LLM
    vector_store=vector_store, #기반 벡터스토어
    similarity_top_k=2, #추후 Retrieve 시 옵션
    mode="collapsed", #Tree구조, Collapse구조 선택
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ],
)

In [ ]:
# 생성된 인덱스 기반 Raptor Retriever 및 쿼리엔진 연결
raptor_retriever =

raptor_query_engine =

In [ ]:
#Naive RAG용
# 청크사이즈 전처리 파이프라인 생성
pipeline_200 = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0)
    ]
)

# 설정한 파이프라인 설정으로 기준 다큐먼트 노드 오브젝트로 변경
nodes_200 = pipeline_200.run(documents=documents)

In [ ]:
# 벡터스토어 인덱스 설정
vector_index_200 = VectorStoreIndex(nodes_200)

# 쿼리 엔진 설정
query_engine_200 = vector_index_200.as_query_engine(similarity_top_k=2)

In [ ]:
# Naive RAG 답안 뽑기:"분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?"
naive_response =

In [ ]:
pprint(naive_response.response)

In [ ]:
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

In [ ]:
# RAPTOR RAG 답안 뽑기:"분당과 같은 신도시가 해외에서도 성공할 수 없는 이유가 뭘까?"
response =

In [ ]:
pprint(response.response)

In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

In [ ]:
# Naive RAG 답안 뽑기:"캐나다가 주목받는 국가인 이유에 대해 알려줘"
naive_response =

In [ ]:
pprint(naive_response.response)

In [ ]:
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

In [ ]:
# RAPTOR RAG 답안 뽑기:"캐나다가 주목받는 국가인 이유에 대해 알려줘"
response =

In [ ]:
pprint(response.response)

In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)

In [ ]:
# Naive RAG 답안 뽑기:"한국과 일본의 유명한 해군 제독에 대해 알려줘"
naive_response =

In [ ]:
pprint(naive_response.response)

In [ ]:
# 참조 컨텍스트 확인
pprint(naive_response.source_nodes[0].text)
pprint(naive_response.source_nodes[1].text)

In [ ]:
# RAPTOR RAG 답안 뽑기:"한국과 일본의 유명한 해군 제독에 대해 알려줘"
response =

In [ ]:
pprint(response.response)

In [ ]:
# 참조 컨텍스트 확인
pprint(response.source_nodes[0].text)
pprint(response.source_nodes[1].text)